# HuggingFace Pretrained GPT2 Feature Extraction on Trn1

## Introduction

This notebook demonstrates how to compile and run a HuggingFace 🤗 Transformers GPT2 model for accelerated feature extraction on Neuron. This notebook will use the [`gpt2`](https://huggingface.co/gpt2) model, which is primarily used for text generation and feature extraction. 

This Jupyter notebook should be run on a Trn1 instance (`trn1.2xlarge` or larger).

## Install Dependencies
This tutorial requires the following pip packages:

- `torch-neuronx`
- `neuronx-cc`
- `transformers`

Most of these packages will be installed when configuring your environment using the Trn1 setup guide. The additional dependencies must be installed here:

In [1]:
# # !pip install "transformers < 4.21.0"
# !pip install git+https://github.com/aws-neuron/transformers-neuronx.git transformers -U

## Compile the model into an AWS Neuron optimized TorchScript

In the following section, we load the model and tokenizer, get s sample input, run inference on CPU, compile the model for Neuron using `torch_neuronx.trace()` and save the optimized model as `TorchScript`.

`torch_neuronx.trace()` expects a tensor or tuple of tensor inputs to use for tracing, so we unpack the tokenzier output. Additionally, the input shape that's used duing compilation must match the input shape that's used during inference. To handle this, we pad the inputs to the maximum size that we will see during inference. We also use define a basic wrapper that ensures the `input_ids` and `attention_mask` kwargs are passed into the GPT2 model in the correct positions without requiring a dictionary. 

In [2]:
from transformers_neuronx.gpt2.model import GPT2ForHuggingFaceSampling


In [2]:
import torch
import torch_neuronx
from transformers import GPT2Tokenizer, GPT2Model


# Create a wrapper to correctly order the inputs
class GPT2Neuron(torch.nn.Module):
    """
    Ensures that `input_ids` and `attention_mask` are passed into the GPT2
    model in the correct positions without requiring a dictionary.
    """

    def __init__(self, model) -> None:
        super().__init__()
        self.model = model

    def forward(self, input_ids, attention_mask):
        return self.model(input_ids=input_ids, attention_mask=attention_mask)


# Create the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token # Define the padding token value
gpt2 = GPT2Model.from_pretrained('gpt2', torchscript=True)
model = GPT2Neuron(gpt2)

model.eval()

# Get an example input
text = "Replace me by any text you'd like."

encoded_input = tokenizer(
    text,
    max_length=128,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)

example = (
    encoded_input['input_ids'],
    encoded_input['attention_mask'],
)

# Run inference on CPU
output_cpu = model(*example)

# Compile the model using the wrapper
model_neuron = torch_neuronx.trace(model, example)

# Save the TorchScript for inference deployment
filename = 'model.pt'
torch.jit.save(model_neuron, filename)

## Run inference and compare results

In this section we load the compiled model, run feature extraction inference on Neuron, and compare the CPU and Neuron outputs.

In [4]:
filename

'model.pt'

In [5]:
example

(tensor([[ 3041,  5372,   502,   416,   597,  2420,   345,  1549,   588,    13,
          50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
          50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
          50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
          50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
          50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
          50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
          50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
          50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
          50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
          50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
          50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
          50256, 50256, 50256, 50256, 50

In [13]:
model_neuron

NeuronModule(original_name=NeuronModule)

In [6]:
# Load the TorchScript compiled model
model_neuron = torch.jit.load(filename)

# Run inference using the Neuron model
output_neuron = model_neuron(*example)

# Compare the results
print(f"CPU outputs:    {output_cpu[0][0][0][:10]}")
print(f"Neuron outputs: {output_neuron[0][0][0][:10]}")

CPU outputs:    tensor([ 0.1629, -0.2166, -0.1410,  0.0061, -0.0623, -0.2181, -0.8142, -0.0920,
        -0.3586,  0.0676], grad_fn=<SliceBackward0>)
Neuron outputs: tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])
